# Advance TVA, with Clustering and Compromising

In [1]:
import warnings

warnings.filterwarnings("ignore")

from src import utils
from src.outcomes import plurality_outcome
from src.outcomes import borda_outcome
from src.happiness_level import HappinessLevel
from src.strategic_voting_risk import StrategicVoting
from src.outcomes import for_two_outcome
from src.Coalition_Compromising import find_stable_coalitions_by_compromising
import os
import numpy as np 
import pandas as pd

In [2]:
path_voting = "input/voting_result.json"
if "env" in os.environ:
    if os.environ["env"] == "docker":
        path_voting = "/input/voting_result.json"

This is the voting input

In [4]:
n_cand = 10
random_voting = utils.random_voting(400, n_cand)
result_random = plurality_outcome(random_voting)
hlevel = HappinessLevel(random_voting, result_random.winner)


In [5]:
borda_result = for_two_outcome(random_voting)
print(borda_result)
voting_df_2= pd.DataFrame(random_voting).T
hlevel = HappinessLevel(random_voting, borda_result.winner)
valori = np.array(list(hlevel.happiness_level_dict.values()))
coals = find_stable_coalitions_by_compromising(100, voting_df_2, valori, borda_result, voting='voting_for_two')

Winner: I
{'A': 73, 'B': 68, 'C': 73, 'D': 86, 'E': 79, 'F': 80, 'G': 85, 'H': 78, 'I': 92, 'J': 86}


UnboundLocalError: cannot access local variable 'new_results' where it is not associated with a value

In [23]:

df = pd.DataFrame(columns=['Number_voters', 'voters','number_manipulations', 'strategic_voting_risk', ])
i=0

dix={}
for el in coals:
    dataset = el[1]
    
    indici = tuple(dataset.index)
    if indici not in list(df['voters']):
        dataset['diff']= dataset['New_H']-dataset['H']
        df.loc[i, 'Number_voters'] = len(indici)
        df.loc[i, 'voters'] = tuple(indici)
        df.loc[i, 'number_manipulations'] = 1
        df.loc[i, 'strategic_voting_risk'] = dataset['diff'].max()
        dix[tuple(indici)]=i
        i+=1
    else:
        dataset['diff']= dataset['New_H']-dataset['H']
        df.loc[dix[tuple(indici)], 'number_manipulations'] += 1
        if dataset['diff'].max() > df.loc[dix[tuple(indici)], 'strategic_voting_risk']:
            df.loc[dix[tuple(indici)], 'strategic_voting_risk'] = dataset['diff'].max()

print(df)





    

   Number_voters                          voters number_manipulations  \
0              6    (62, 63, 214, 216, 223, 305)                    1   
1              5       (187, 204, 241, 248, 268)                    1   
2              4             (42, 109, 138, 384)                    1   
3              6  (136, 155, 170, 325, 334, 399)                    1   
4              4            (209, 280, 308, 393)                    1   
5              6      (23, 29, 36, 99, 104, 319)                    2   
6              2                      (231, 338)                    2   
7              5        (47, 125, 274, 311, 352)                    1   
8              3                  (82, 103, 122)                    1   
9              2                      (188, 296)                    2   
10             2                      (100, 318)                    1   
11             3                 (117, 370, 374)                    1   
12             2                      (182, 356)   